In [ ]:
import socket
import threading

#Los usuarios se guardan en 'application_users.txt' sabiendo que en cada linea del txt hay únicamente un usuario con su contraseña
#siguiendo la estructura usuario:contraseña

# Ruta al archivo de usuarios
users_file = 'application_users.txt'
connections_file='current_connections.txt'


# Función para comprobar si un usuario ya existe
def user_exists(username): #Función que busca si un usuario ya está guardado.
    with open(users_file, 'r') as file:
        for line in file:
            if username == line.strip().split(':')[0]: #Se busca linea por linea si ecxiste alguna coincidencia
                return True # Si se encuentra una coincidencia se sale del bucle y se devuelve True
    return False #Si no se encuentra ninguna coincidencia se sale con un False

# Función para registrar un nuevo usuario
def register_user(username, password):
    if user_exists(username):#Se checkea paar que no haya nombres duplicados
        return False
    else:
        with open(users_file, 'a') as file:
            file.write(f'{username}:{password}\n') # Si no hay un duplicado se añade una nueva linea con username:password
        return True

# Función para validar el login de un usuario
def login_user(username, password):
    with open(users_file, 'r') as file:
        for line in file: 
            user, passw = line.strip().split(':')
            if username == user and password == passw:  # Para cada linea se compara si la combinacion de usuario 
                                                        # mas contraseña existe en alguna línea
                return True                             # si existe alguna coincidencia se hace login.
    return False                                        # si no existe alguna coincidencia no se hace login.

def add_connection(username, client_address):
    with open(connections_file, 'a') as file:
        file.write(f'{username}:{client_address}\n') # Si no hay un duplicado se añade una nueva linea con username:password
    return True
def remove_connection(username, client_address):
    with open(connections_file, 'r') as file:
        lines = file.readlines()  # Leer todas las líneas del archivo
    
    # Filtrar las líneas para eliminar la correspondiente a la conexión que queremos borrar
    with open(connections_file, 'w') as file:
        for line in lines:
            if line.strip() != f'{username}:{client_address}':
                file.write(line)
    return True
    
# Manejo de conexiones de cliente
def handle_client(connection, client_address):
    error_message=''
    try:
        while True:  # Mantener la conexión abierta
            data = connection.recv(1024).decode()
            if not data:
                break  # Romper el bucle si no se recibe data
            elif len(data.split(" ")) == 1:
                command= data
            else:
                command, username, password = data.split(' ')
            
            if command == 'exit':
                remove_connection(username, client_address)
                break  # Romper el bucle si el comando es 'exit'
            elif command=='testconnection':
                connection.sendall('Usted sigue conectado al servidor'.encode())
            elif command == 'register':
                if register_user(username, password):
                    connection.sendall('Registro exitoso'.encode())
                else:
                    error_message='Error: el usuario ya existe'
                    break
            elif command == 'login':
                if login_user(username, password):
                    add_connection(username, str(client_address))
                    connection.sendall('Login exitoso'.encode())
                else:
                    error_message='Error: usuario o contraseña incorrectos'
                    break
            else:
                break
    finally:
        connection.sendall(('Desconectado. Se le ha desconectado del servidor'+ error_message).encode())
        connection.close()
        

# Configuración del servidor
# Configuración del servidor
server_address = ('localhost', 6780)
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind(server_address)
sock.listen()

print("Servidor iniciado, esperando conexiones...")

while True:
    connection, client_address = sock.accept()
    print(f"Conexión desde {client_address}")
    threading.Thread(target=handle_client, args=(connection, client_address)).start()

Servidor iniciado, esperando conexiones...


Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\Users\Alfredo\anaconda3\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\Alfredo\anaconda3\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Alfredo\AppData\Local\Temp\ipykernel_32920\1869256640.py", line 79, in handle_client
UnboundLocalError: local variable 'username' referenced before assignment


Conexión desde ('127.0.0.1', 56687)


Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\Alfredo\anaconda3\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Users\Alfredo\anaconda3\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Alfredo\AppData\Local\Temp\ipykernel_32920\1869256640.py", line 79, in handle_client
UnboundLocalError: local variable 'username' referenced before assignment


Conexión desde ('127.0.0.1', 56689)
